# Create context-specific models - Mouse G6PD variants omics data
## Setup
### Import packages

In [ ]:
import re
from collections import defaultdict

import gurobipy as gp
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib_venn as mpl_venn
import numpy as np
import pandas as pd
import seaborn as sns
import sympy
from rbc_gem_utils import (
    COBRA_CONFIGURATION,
    get_dirpath,
    read_cobra_model,
    show_versions,
    write_cobra_model,
)
from rbc_gem_utils.analysis.overlay import (
    DEFAULT_PREFIX_SUFFIX_VALUES,
    ProteinDilution,
    add_relaxation_budget,
    load_overlay_model,
    update_slack_value,
)
from rbc_gem_utils.util import AVOGADRO_NUMBER, DEFAULT_DRY_MASS_PER_CELL
from sklearn.metrics import r2_score

gp.setParam("OutputFlag", 0)
gp.setParam("LogToConsole", 0)

# Show versions of notebook
show_versions()

### Define configuration
#### COBRA Configuration

In [ ]:
COBRA_CONFIGURATION.solver = "gurobi"
# Set bound defaults much larger to prevent model loading issues
COBRA_CONFIGURATION.bounds = (-1e-8, 1e8)
COBRA_CONFIGURATION.tolerance = 1e-7
COBRA_CONFIGURATION

### Define organism, model, and dataset

In [ ]:
organism = "Mouse"
model_id = "RBC_GEM"
dataset_name = "G6PDvariants"

### Set variables for columns keys and sample identification

In [ ]:
sample_key = "SAMPLE ID"
donor_key = "MOUSE ID"
time_key = "TIME"

timepoints = ["Pre", "Post", "TD"]
phenotypes = ["HumCan", "A", "MED"]

# For sample IDs
donor_re = re.compile(rf"(?P<donor>({'|'.join(phenotypes)})\d+)")
time_re = re.compile(rf"(?P<time>{'|'.join(timepoints)})")
phenotype_re = re.compile(rf"(?P<phenotype>({'|'.join(phenotypes)}))")

operations = "|".join([x.capitalize() for x in ["mean", "median"]])

operation_re = re.compile(r"(?P<op>" + operations + r")\_(?P<group>\w+)")
sample_id_re = re.compile(
    r"(?!" + operations + r")" + donor_re.pattern + r"\_" + time_re.pattern
)

### Set computation options

In [ ]:
run_computations = False
overwrite = False
verbose = True
objective_reactions = ["NaKt"]

### Set figure options

In [ ]:
save_figures = True
transparent = False
imagetype = "svg"

### Set paths

In [ ]:
# Set paths
processed_data_dirpath = get_dirpath(use_temp="processed") / organism / dataset_name
overlay_dirpath = get_dirpath("analysis") / "OVERLAY" / organism
model_dirpath = overlay_dirpath / model_id

results_dirpath = (
    get_dirpath(use_temp="processed") / model_id / "OVERLAY" / organism / dataset_name
)

fitting_dirpath = results_dirpath / "fitting"
sample_pcmodels_dirpath = results_dirpath / "sample_pcmodels"
# Ensure directories exist
results_dirpath.mkdir(exist_ok=True, parents=True)
fitting_dirpath.mkdir(exist_ok=True, parents=True)
sample_pcmodels_dirpath.mkdir(exist_ok=True, parents=True)

### Define hemoglobin proteins

In [ ]:
# Identify hemoglobin proteins
HB_PROTEINS = {
    k.replace("-", "_"): v
    for k, v in {
        "Hba": "P01942",  # Hemoglobin subunit alpha
        "Hba-a1": "P01942",
        "Hbb-b1": "P02088",  # Hemoglobin subunit beta-1
        "Hbb-b2": "P02089",  # Hemoglobin subunit beta-2
        "Hbb-bh0": "P04443",  # Hemoglobin subunit beta-H0
        "Hbb-bh1": "P04444",  # Hemoglobin subunit beta-H1
        "Hbz": "P06467",  # Hemoglobin subunit zeta
        "Hba-x": "P06467",
        "Hbz1": "P06467",
        "Hbb-y": "P02104",  # Hemoglobin subunit epsilon-Y2
    }.items()
}

## Load RBC-GEM models

In [ ]:
# Load models
model = read_cobra_model(filename=model_dirpath / f"{model_id}.xml")
pcmodel = load_overlay_model(filename=model_dirpath / f"{model_id}_PC.xml")

pcmodel

## Load copy numbers and protein data

In [ ]:
# Load protein copy numbers
df_copy_numbers = pd.read_csv(
    processed_data_dirpath / f"{dataset_name}_ProteinCopyNumbers.csv",
    index_col=sample_key,
)

# Load MCH for transforming data
df_MCH = pd.read_csv(
    processed_data_dirpath / f"{dataset_name}_MCH.csv", index_col=sample_key
)
# Load intensity data as weighting matrix
df_weighting_mat = pd.read_csv(
    processed_data_dirpath / f"{dataset_name}_ProteinIntensities.csv",
    index_col=sample_key,
)

# Load protein data
df_protein_data = pd.read_csv(
    processed_data_dirpath / f"{dataset_name}_ProteinData.csv",
    index_col="Entry",
)

all_ids = list(df_copy_numbers.index.unique())
operation_ids = [x for x in all_ids if operation_re.match(x)]
sample_ids = [x for x in all_ids if sample_id_re.match(x)]

print(f"Number of measured samples: {len(sample_ids)}")
print(f"Number of operation samples: {len(operation_ids)}")
print(f"Number of models to generate: {len(all_ids)}")
print(f"\nNumber of timepoints across samples: {len(timepoints)}")

## Integrate proteomics with model
### Scale measurements for proteome budget
Note that this step will help ensure its theoretically possible for a perfect fit 

In [ ]:
HB_PERCENT, LA_PERCENT = (0.95, 0.05)
MODELED_PERCENT = HB_PERCENT + LA_PERCENT
assert 1 >= MODELED_PERCENT

budget_hb_value = 1000 * HB_PERCENT
budget_la_value = 1000 * LA_PERCENT
budget_total_value = 1000 * MODELED_PERCENT

print(f"Hemoglobin budget:\t{budget_hb_value} mg protein / gDW")
print(f"Low abundance budget:\t{budget_la_value} mg protein / gDW")
print(f"Total budget:\t\t{budget_total_value} mg protein / gDW")

In [ ]:
df_uniprot_to_mw = df_protein_data["Mass"].astype(float)

gDW_total_protein = (
    df_MCH  # pgDW HB
    * (1 / HB_PERCENT)  # pgDW total protein / pgDW HB
    * (1 / 1e12)  #  gDW total protein / pgDW total protein
)
gDW_total_protein

# protein copies / cell
df_mg_prot_per_gDW = (
    df_copy_numbers[df_protein_data.index].mul(
        1 / gDW_total_protein.squeeze(), axis=0  # cell / gDW
    )
    * (
        1
        / AVOGADRO_NUMBER  # mol / protein copies
        * df_uniprot_to_mw  # g / mol
        * 1e3  # mg / g
    ).copy()
)
df_mg_prot_per_gDW = df_mg_prot_per_gDW.loc[df_copy_numbers.index]
df_mg_prot_per_gDW[df_mg_prot_per_gDW.index.duplicated()]

In [ ]:
# Split into hemoglobin and low abundance proteomes
budget_rxn_prefix = DEFAULT_PREFIX_SUFFIX_VALUES["budgets"]["prefix.dilution"]
budget_met_prefix = DEFAULT_PREFIX_SUFFIX_VALUES["budgets"]["prefix.metabolite"]
df_mg_prot_per_gDW_hb = df_mg_prot_per_gDW.loc[
    :, df_mg_prot_per_gDW.columns.isin(HB_PROTEINS.values())
]
df_mg_prot_per_gDW_la = df_mg_prot_per_gDW.loc[
    :, ~df_mg_prot_per_gDW.columns.isin(HB_PROTEINS.values())
]

df_summary = {
    "Perfect total": 1000,
    "Current total": df_mg_prot_per_gDW.loc[sample_ids].sum(axis=1).mean().item(),
    "Hemoglobin total": df_mg_prot_per_gDW_hb.loc[sample_ids].sum(axis=1).mean().item(),
    "Low abundance total": df_mg_prot_per_gDW_la.loc[sample_ids]
    .sum(axis=1)
    .mean()
    .item(),
}
df_summary["Remaining/Excess"] = df_summary["Perfect total"] - (
    df_summary["Hemoglobin total"] + df_summary["Low abundance total"]
)

budget_rxn_proteome = pcmodel.reactions.get_by_id(
    f"{budget_rxn_prefix}{budget_met_prefix}proteome"
)
budget_rxn_hemoglobin = pcmodel.reactions.get_by_id(
    f"{budget_rxn_prefix}{budget_met_prefix}hemoglobin"
)
budget_rxn_total = pcmodel.reactions.get_by_id(
    f"{budget_rxn_prefix}{budget_met_prefix}total"
)

if budget_la_value is None:
    budget_la_value = budget_rxn_proteome.upper_bound
if budget_hb_value is None:
    budget_hb_value = budget_rxn_hemoglobin.upper_bound
if budget_total_value is None:
    budget_total_value = budget_rxn_total.upper_bound

assert budget_total_value >= (budget_la_value + budget_hb_value)

budget_rxn_proteome.upper_bound = budget_la_value
budget_rxn_hemoglobin.upper_bound = budget_hb_value
budget_rxn_total.upper_bound = budget_total_value

# Scale values for low abundance proteome
budget_value = budget_la_value
df_mg_prot_per_gDW_la = (
    budget_value * (df_mg_prot_per_gDW_la.T / df_mg_prot_per_gDW_la.sum(axis=1)).T
)
df_summary["Low abundance scaled"] = budget_value

# Scale values for hemoglobin proteome
budget_value = budget_hb_value
df_mg_prot_per_gDW_hb = (
    budget_value * (df_mg_prot_per_gDW_hb.T / df_mg_prot_per_gDW_hb.sum(axis=1)).T
)
df_summary["Hemoglobin scaled"] = budget_value

budget_value = budget_total_value - sum([budget_la_value, budget_hb_value])
df_summary["Remaining scaled"] = budget_value

# Combine dataframes back into one
df_mg_prot_per_gDW_normalized = pd.concat(
    (df_mg_prot_per_gDW_hb, df_mg_prot_per_gDW_la), axis=1
)
df_summary = pd.DataFrame.from_dict(
    {
        " " * max(30 - len(k), 0) + k: [f"{v:.4f}", f"{v / 1000 * 100:.1f}%"]
        for k, v in df_summary.items()
    },
    orient="index",
    columns=["mg protein / gDW / cell", "Percentage"],
)
print(df_summary)
df_mg_prot_per_gDW_normalized.sum(axis=1)

### Convert mg / gDW to nmol / gDW

In [ ]:
df_nmol_prot_per_gDW = (
    df_mg_prot_per_gDW_normalized  # mg / gDW
    * (1 / df_uniprot_to_mw)  # mol / g --> mmol / mg
    * (1e6 / 1)  # nmol / mmol
).loc[:, df_mg_prot_per_gDW_normalized.columns]
df_nmol_prot_per_gDW = df_nmol_prot_per_gDW.T
df_nmol_prot_per_gDW

## Create DataFrame for protein dilution reactions

In [ ]:
df_model_protein_dilutions = pd.concat(
    (
        pd.Series(
            {g.annotation.get("uniprot"): g.id for g in model.genes}, name="genes"
        ),
        pd.Series(
            {
                protdl.annotation.get("uniprot"): protdl.id
                for protdl in pcmodel.reactions.query(
                    lambda x: isinstance(x, ProteinDilution)
                )
            },
            name="PROTDL",
        ),
    ),
    axis=1,
).dropna()
df_model_protein_dilutions.index.name = "uniprot"
df_model_protein_dilutions = df_model_protein_dilutions[
    df_model_protein_dilutions["genes"].isin(model.genes.list_attr("id"))
].sort_values("PROTDL")
df_model_protein_dilutions

## Organize samples (optional)
Use this for organizing samples if time-outs are an issue or multiple runs are necessary

In [ ]:
df_samples = df_nmol_prot_per_gDW.copy()
df_samples

### Map samples to model

In [ ]:
merge_key = "uniprot"
df_samples.index.name = merge_key

df_model = (
    df_model_protein_dilutions[["PROTDL"]]
    .merge(df_samples, left_index=True, right_index=True, how="left")
    .set_index("PROTDL")
    .sort_index()
)
no_experimental_measurements = [
    protein_dilution
    for protein_dilution, has_measurement in df_model.isna().all(axis=1).items()
    if has_measurement
]
print(
    f"Model proteins mapped to measurements: {len(df_model) - len(no_experimental_measurements)}"
)
print(f"Model proteins without measurements: {len(no_experimental_measurements)}")
df_model[~df_model.isna().all(axis=1)]

#### Summarize mapping

In [ ]:
dataset_proteins = set(df_samples.index)
model_proteins = set(df_model_protein_dilutions.index)

df_mg_prot_per_gDW_hb = df_mg_prot_per_gDW_normalized.loc[
    [
        x for x in df_mg_prot_per_gDW_normalized.index if x in sample_ids
    ],  # Don't include operation IDs
    [
        x
        for x in df_mg_prot_per_gDW_normalized.columns
        if x in list(HB_PROTEINS.values())
    ],
]
df_mg_prot_per_gDW_la = df_mg_prot_per_gDW_normalized.loc[
    [
        x for x in df_mg_prot_per_gDW_normalized.index if x in sample_ids
    ],  # Don't include operation IDs
    [x for x in df_mg_prot_per_gDW.columns if not x in list(HB_PROTEINS.values())],
]

df_mapped_mass_la = df_mg_prot_per_gDW_la.loc[
    :, df_mg_prot_per_gDW_la.columns.isin(model_proteins)
].sum(axis=1)
df_unmapped_mass_la = df_mg_prot_per_gDW_la.loc[
    :, ~df_mg_prot_per_gDW_la.columns.isin(model_proteins)
].sum(axis=1)
df_mapped_mass_hb = df_mg_prot_per_gDW_hb.loc[
    :, df_mg_prot_per_gDW_hb.columns.isin(model_proteins)
].sum(axis=1)
df_unmapped_mass_hb = df_mg_prot_per_gDW_hb.loc[
    :, ~df_mg_prot_per_gDW_hb.columns.isin(model_proteins)
].sum(axis=1)

proteomes = {}
round_int = 6
for label, df in zip(
    ["hemoglobin", "low abundance"], [df_mg_prot_per_gDW_hb, df_mg_prot_per_gDW_la]
):
    df_modeled = df.loc[:, df.columns.isin(model_proteins)].sum(axis=1)
    df_remaining = df.loc[:, ~df.columns.isin(model_proteins)].sum(axis=1)
    means = (df_modeled.mean(), df_remaining.mean())
    stdevs = (df_modeled.std(), df_remaining.std())
    proteomes[(label, "modeled")] = round(means[0], round_int)
    proteomes[(label, "remaining")] = round(means[1], round_int)
proteomes = pd.Series(proteomes, name="Mean value across samples")
proteomes.index = [f"Mean {k[0]} mass {k[1]}" for k in proteomes.index]
print(proteomes.head())
proteomes = proteomes[proteomes != 0]

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(3, 6))
subsets = (
    len(dataset_proteins),
    len(model_proteins),
    len(dataset_proteins.intersection(model_proteins)),
)


venn = mpl_venn.venn2(
    subsets=subsets,
    set_labels=(dataset_name, model.id),
    set_colors=("red", "blue"),
    alpha=0.5,
    ax=ax1,
)
circles = mpl_venn.venn2_circles(
    subsets=subsets, linestyle="-", color="black", ax=ax1, linewidth=1
)
for text in venn.set_labels:
    text.set_fontsize("x-large")
for text in venn.subset_labels:
    text.set_fontsize("x-large")
ax1.set_title("Modeled proteome", fontsize="xx-large")


label_color_map = {
    "Mean hemoglobin mass modeled": ("Hemoglobin", "xkcd:dark red"),
    "Mean low abundance mass modeled": ("Low abundance", "xkcd:light blue"),
    "Mean low abundance mass remaining": ("Not modeled", "xkcd:green"),
}
edgecolor = "black"
linewidth = 1
ax2.pie(
    x=proteomes.values,
    colors=[label_color_map[k][1] for k in proteomes.index],
    pctdistance=1.35,
    counterclock=False,
    autopct=lambda pct: f"{pct * 1000/100:.2f}\n",
    textprops=dict(fontsize="large", ha="center", va="top"),
    wedgeprops=dict(edgecolor=edgecolor, linewidth=linewidth),
)
handles = [
    mpl.patches.Patch(
        edgecolor=edgecolor,
        linewidth=linewidth,
        label=label_color_map[k][0],
        facecolor=label_color_map[k][1],
    )
    for k in proteomes.index
]
ax2.legend(
    handles=handles,
    ncols=1,
    bbox_to_anchor=(0.5, 0),
    loc="upper center",
    fontsize="large",
    frameon=False,
)
ax2.set_xlabel("Mass (mg/gDW)", fontsize="large", labelpad=-10)
if save_figures:
    fig.savefig(
        results_dirpath / f"ModeledProteome.{imagetype}",
        transparent=transparent,
        format=imagetype,
    )
fig;

## Create QP model for each sample

In [ ]:
def solve_qp(pcmodel, df):
    x = []  # Variables
    c = []  # Data * Weights
    F = []  # Weights

    for protdl, (data_value, weight) in df.iterrows():
        protdl = pcmodel.reactions.get_by_id(protdl)
        x.append(protdl.flux_expression)
        c.append(weight * data_value)
        F.append(weight)

    x = sympy.Matrix(x)
    c = sympy.Matrix(c)
    F = sympy.DiagMatrix(sympy.Matrix(F))
    # # QP Objective must be in form of 0.5 * x.T * F * x - c.T * x
    objective = 0.5 * x.T * F * x - c.T * x
    pcmodel.objective = objective[0]
    pcmodel.objective_direction = "min"
    pcmodel.tolerance = 1e-9

    qp_sol = pcmodel.optimize()
    return qp_sol


def solve_qp_for_samples(
    pcmodel, df_samples, df_weights=None, log_zero_replacement=1e-6, verbose=False
):
    qp_solutions_dict = {}
    for sample_id, data_series in df_samples.items():
        # Get protein values
        data_series.name = "Data"
        if df_weights is None:
            data_weights = 1 / data_series.replace(0, 1)
            data_weights = data_weights / data_weights.mean()
        else:
            data_weights = df_weights.loc[:, sample_id]
        # Get protein weights
        data_weights.name = "Weights"

        # Map to model, currently model mapping DataFrame generated outside scope of function
        df_model_data_weights = (
            df_model_protein_dilutions[["PROTDL"]]
            .merge(data_series, left_index=True, right_index=True, how="left")
            .merge(data_weights, left_index=True, right_index=True, how="left")
            .set_index("PROTDL")
            .sort_index()
        )

        df = (
            df_model_data_weights.loc[:, [data_series.name, data_weights.name]]
            .dropna(axis=0, how="all")
            .astype(float)
        )

        with pcmodel:
            qp_sol = solve_qp(pcmodel, df)

        df_qp_sol = qp_sol.fluxes.loc[
            pcmodel.reactions.query(lambda x: isinstance(x, ProteinDilution)).list_attr(
                "id"
            )
        ]
        df_qp_sol = (
            pd.concat((df_model_data_weights, df_qp_sol), axis=1).dropna().sort_index()
        )
        # data_weights = df_qp_sol.loc[:, "Weights"]

        df_qp_sol = df_qp_sol.rename(
            {"Data": "Measured Proteome", "fluxes": "Best-Fitted Proteome"}, axis=1
        )
        df_qp_sol = df_qp_sol.loc[:, ["Measured Proteome", "Best-Fitted Proteome"]]

        df = df_qp_sol.copy()
        r2 = r2_score(
            df.iloc[:, 0].values, df.iloc[:, 1].values, multioutput="uniform_average"
        )

        df = df_qp_sol.apply(
            lambda x: [
                log_zero_replacement if np.isclose(y, 0, atol=1e-12) else y for y in x
            ]
        ).apply(np.log10)
        r2_log10_w_outliers = r2_score(
            df.iloc[:, 0].values, df.iloc[:, 1].values, multioutput="uniform_average"
        )

        df = df_qp_sol[
            ~df_qp_sol.apply(lambda x: np.isclose(x, 0).any(), axis=1)
        ].apply(np.log10)
        r2_log10_wo_outliers = r2_score(
            df.iloc[:, 0].values, df.iloc[:, 1].values, multioutput="uniform_average"
        )
        r2_values = (r2, r2_log10_w_outliers, r2_log10_wo_outliers)
        qp_solutions_dict[sample_id] = (df_qp_sol, qp_sol.objective_value, r2_values)
        if verbose:
            # Recall that the objective is designed to try to minimize fitting error via maximizing R2, so 1 is a possibility
            print(
                "Sample '{}'\tR^2: {:.4f}\tR^2 log10 w/outliers: {:.4f}\tR^2 log10 wo/outliers: {:.4f}\tObjective: {:.5f}".format(
                    sample_id, *r2_values, qp_sol.objective_value
                )
            )
        # TODO catch bad fits

    return qp_solutions_dict

### Set weightings for QP problem

In [ ]:
# Ensure data is provided as (Protein IDs x Sample IDs)
# Use original copy number values for weights
df_weights = df_weighting_mat.T.loc[df_protein_data.index, df_samples.columns]
df_weights = 1 / df_weights.infer_objects(copy=False).replace(0, 1)
df_weights /= df_weights.mean()
df_weights

### Fit data by solving QP

In [ ]:
log_zero_replacement = 1e-9
fitting_data = {"measured": {}, "best_fit": {}, "r2_objective": {}}
if run_computations:
    qp_solutions_dict = solve_qp_for_samples(
        pcmodel,
        df_samples,
        df_weights=df_weights,
        log_zero_replacement=log_zero_replacement,
        verbose=True,
    )

    for sample_id, (
        df_qp_sol,
        objective_value,
        r2_values,
    ) in qp_solutions_dict.items():
        fitting_data["measured"][sample_id] = df_qp_sol["Measured Proteome"].to_dict()
        fitting_data["best_fit"][sample_id] = df_qp_sol[
            "Best-Fitted Proteome"
        ].to_dict()
        fitting_data["r2_objective"][sample_id] = {
            "Objective": objective_value,
            "R^2": r2_values[0],
            "R^2 log10 w/ outliers": r2_values[1],
            "R^2 log10 w/o outliers": r2_values[2],
        }
    for key, data in fitting_data.items():
        data = pd.DataFrame.from_dict(data, orient="columns")
        data.to_csv(fitting_dirpath / f"proteome_{key}.csv", index=True)
        fitting_data[key] = data
else:
    for key in fitting_data.keys():
        fitting_data[key] = pd.read_csv(
            fitting_dirpath / f"proteome_{key}.csv", index_col=0
        )
    qp_solutions_dict = {}
    for sample_id in df_samples.columns:
        df_qp_sol = pd.concat(
            (
                fitting_data["measured"].loc[:, sample_id],
                fitting_data["best_fit"].loc[:, sample_id],
            ),
            axis=1,
        )
        df_qp_sol.columns = ["Measured Proteome", "Best-Fitted Proteome"]
        r2_values = fitting_data["r2_objective"].loc[:, sample_id].values
        objective_value, r2_values = r2_values[0], r2_values[1:]
        qp_solutions_dict[sample_id] = (df_qp_sol, objective_value, r2_values)
print(f"Number of QP solutions: {len(qp_solutions_dict)}")

### Plot fitting 
#### For the mean and median samples of each time point and for each chosen phenotype

In [ ]:
samples_to_plot = np.array(
    [
        # Best for 1 or 3 columns
        [x for x in operation_ids if (timepoint in x and phenotype in x)]
        for phenotype in phenotypes
        for timepoint in timepoints
    ]
).T
samples_to_plot

In [ ]:
r2_text_loc = "upper left"
transform = False

length = 4
nrows, ncols = samples_to_plot.shape
fig, axes = plt.subplots(
    nrows=nrows,
    ncols=ncols,
    figsize=(length * ncols, length * nrows),
    sharex=True,
    sharey=True,
)
sns.despine(fig)
for idx, (sample_id, ax) in enumerate(zip(samples_to_plot.flatten(), axes.flatten())):
    df_qp_sol, objective_value, r2_values = qp_solutions_dict[sample_id]
    # Copy to prevent alterations to the original
    df_qp_sol = df_qp_sol.copy()
    xlabel, ylabel = df_qp_sol.columns

    ticks = 10 ** np.arange(*np.log10([log_zero_replacement, 1e8]), 3)
    if transform:
        ticks = np.log10(ticks)
        df_qp_sol.iloc[:, 0] = (
            df_qp_sol.iloc[:, 0]
            .apply(lambda x: log_zero_replacement if np.isclose(x, 0) else x)
            .apply(np.log10)
        )
        df_qp_sol.iloc[:, 1] = (
            df_qp_sol.iloc[:, 1]
            .apply(lambda x: log_zero_replacement if np.isclose(x, 0) else x)
            .apply(np.log10)
        )
    perfect_fit_line = ax.plot(
        [ticks[0], ticks[-1]],
        [ticks[0], ticks[-1]],
        linestyle=":",
        color="black",
        linewidth=1,
        alpha=1,
    )

    zero_val = 0 if not transform else np.log10(log_zero_replacement)

    df_zeros = df_qp_sol[
        (df_qp_sol.apply(lambda x: np.isclose(x, zero_val))).any(axis=1)
    ]
    df_perfect = df_qp_sol[
        np.isclose(
            abs(df_qp_sol["Measured Proteome"] - df_qp_sol["Best-Fitted Proteome"]), 0
        )
    ]
    df_perfect = df_perfect[~df_perfect.index.isin(df_zeros.index)]

    df_altered = df_qp_sol[
        ~np.isclose(
            abs(df_qp_sol["Measured Proteome"] - df_qp_sol["Best-Fitted Proteome"]), 0
        )
    ]
    df_altered = df_altered[~df_altered.index.isin(df_zeros.index)]
    df_always_zero = df_zeros[(df_zeros == zero_val).all(axis=1)]
    df_zeros = df_zeros[~df_zeros.index.isin(df_always_zero.index)]
    df_from_zeros = df_zeros[np.isclose(df_zeros["Measured Proteome"], zero_val)]
    df_to_zeros = df_zeros[np.isclose(df_zeros["Best-Fitted Proteome"], zero_val)]

    handles = [
        ax.scatter(
            data=df_perfect.replace(0, ticks[0]),
            x=xlabel,
            y=ylabel,
            color="xkcd:blue",
            alpha=0.5,
            edgecolors="black",
            linewidths=1,
        ),
        ax.scatter(
            data=df_altered.replace(0, ticks[0]),
            x=xlabel,
            y=ylabel,
            color="xkcd:yellow",
            alpha=0.5,
            edgecolors="black",
            linewidths=1,
        ),
        ax.scatter(
            data=df_from_zeros.replace(0, ticks[0]),
            x=xlabel,
            y=ylabel,
            color="xkcd:green",
            alpha=0.5,
            edgecolors="black",
            linewidths=1,
        ),
        ax.scatter(
            data=df_to_zeros.replace(0, ticks[0]),
            x=xlabel,
            y=ylabel,
            color="xkcd:red",
            alpha=0.5,
            edgecolors="black",
            linewidths=1,
        ),
        ax.scatter(
            data=df_always_zero.replace(0, ticks[0]),
            x=xlabel,
            y=ylabel,
            color="xkcd:black",
            alpha=0.5,
            edgecolors="black",
            linewidths=1,
        ),
    ]
    labels = [
        f"Perfect fit",
        f"Adjusted abundance",
        f"Unexpressed to expressed",
        f"Expressed to unexpressed",
        f"Never expressed",
    ]

    op, time, phenotype = str(sample_id.replace(f"{pcmodel.id}_", "")).split("_")
    sample_label = " ".join((op, time, phenotype))

    if not transform:
        ax.set_xscale("log")
        ax.set_yscale("log")

    fontdict = {"size": "xx-large"}
    if idx >= len(samples_to_plot.flatten()) - ncols:
        ax.set_xlabel(xlabel, fontdict=fontdict)
    fig.legend(
        handles=handles,
        labels=labels,
        loc="upper center",
        ncols=len(labels),
        frameon=False,
        fontsize="large",
        markerscale=2,
        bbox_to_anchor=(0.5, -0.01),
    )
    if idx % ncols == 0:
        ax.set_ylabel(ylabel, fontdict=fontdict)

    ax.set_xticks(ticks)
    ax.set_yticks(ticks)

    ax.xaxis.set_tick_params(labelsize="x-large")
    ax.yaxis.set_tick_params(labelsize="x-large")

    r2_format = " = {:.4f}"
    if r2_text_loc == "lower right":
        ax.text(
            0.95,
            0.1,
            "\n".join(
                (
                    sample_label,
                    r"$R^{2}$" + r2_format.format(r2_values[0]),
                    r"$R^{2}\text{log}_{10}\text{ w/  outliers}$"
                    + r2_format.format(r2_values[1]),
                    r"$R^{2}\text{log}_{10}\text{ w/o outliers}$"
                    + r2_format.format(r2_values[2]),
                )
            ),
            transform=ax.transAxes,
            color="black",
            fontsize="medium",
            ha="right",
        )
    elif r2_text_loc == "upper left":
        ax.text(
            0.05,
            0.8,
            "\n".join(
                (
                    sample_label,
                    r"$R^{2}$" + r2_format.format(r2_values[0]),
                    r"$R^{2}\text{log}_{10}\text{ w/  outliers}$"
                    + r2_format.format(r2_values[1]),
                    r"$R^{2}\text{log}_{10}\text{ w/o outliers}$"
                    + r2_format.format(r2_values[2]),
                )
            ),
            transform=ax.transAxes,
            color="black",
            fontsize="medium",
            ha="left",
        )
    else:
        pass
fig.tight_layout()
if save_figures:
    fig.savefig(
        fitting_dirpath
        / f"QPfitting_{'' if not transform else 'log10_'}{model_id}.{imagetype}",
        transparent=transparent,
        format=imagetype,
    )

### Determine best value for slack variable

In [ ]:
slack_determination_models = samples_to_plot.flatten()
list_of_pcmodels = []
for sample_id in slack_determination_models:
    df_qp_sol, objective_value, r2_values = qp_solutions_dict[sample_id]
    # Create a copy of the model
    pcmodel_sample = pcmodel.copy()
    pcmodel_sample.id = f"{pcmodel.id}_{sample_id}"
    for protdl in pcmodel_sample.reactions.query(
        lambda x: isinstance(x, ProteinDilution)
    ):
        if protdl.id in df_qp_sol.index:
            prot_bound = df_qp_sol.loc[protdl.id]["Best-Fitted Proteome"]
        else:
            prot_bound = 0
        protdl.bounds = (float(prot_bound), float(prot_bound))
    # Add the relaxation budget with slack = 0 first
    add_relaxation_budget(pcmodel_sample, 0, int(verbose))
    list_of_pcmodels += [pcmodel_sample]

In [ ]:
slack_min = 1e-5  # Slack %
slack_max = 1.5
if run_computations:
    solutions = {
        pcmodel_sample.id: defaultdict(list) for pcmodel_sample in list_of_pcmodels
    }
    for slack_value in np.geomspace(slack_min, slack_max, 101):
        if int(verbose):
            print(f"Updating slack variable to {slack_value:.4%}.")
        for pcmodel_sample in list_of_pcmodels:
            update_slack_value(pcmodel_sample, slack_value, verbose=False)
            budget_rxn_relaxation = pcmodel_sample.reactions.get_by_id(
                f"{budget_rxn_prefix}{budget_met_prefix}relaxation"
            )
            pcmodel_sample.objective = (
                sum(
                    [
                        r.flux_expression
                        for r in pcmodel_sample.reactions.get_by_any(
                            objective_reactions
                        )
                    ]
                )
                - budget_rxn_relaxation.flux_expression
            )
            pcmodel_sample.objective_direction = "max"
            sol = pcmodel_sample.optimize()
            obj_value = sol.objective_value
            if not obj_value or np.isnan(obj_value):
                if int(verbose) > 1:
                    print(f"No solution for {slack_value:.4%}%\n.")
                continue
            else:
                demand = budget_rxn_relaxation.flux
                budget = budget_rxn_relaxation.upper_bound
            solutions[pcmodel_sample.id]["model"].append(pcmodel_sample.id)
            solutions[pcmodel_sample.id]["slack"].append(slack_value)
            solutions[pcmodel_sample.id]["objective"].append(obj_value)
            solutions[pcmodel_sample.id]["_".join(objective_reactions)].append(
                obj_value + demand
            )
            solutions[pcmodel_sample.id]["relaxation"].append(demand / budget)
    solutions = {
        pcmodel_sample_id: pd.DataFrame.from_dict(sol)
        for pcmodel_sample_id, sol in solutions.items()
    }
    df_relaxation = pd.concat(list(solutions.values()), axis=0)
    df_relaxation.to_csv(
        fitting_dirpath / f"SlackPercentDeterminationData_{model_id}.csv",
        index=False,
    )
else:
    df_relaxation = pd.read_csv(
        fitting_dirpath / f"SlackPercentDeterminationData_{model_id}.csv",
    )
    solutions = {
        mid: df_relaxation[df_relaxation["model"] == mid].drop("model", axis=1)
        for mid in df_relaxation["model"].unique()
    }
df_relaxation

In [ ]:
chosen_slack_var = 0.03
colors = {
    "Pre_HumCan": "xkcd:light green",
    "Pre_MED": "xkcd:light blue",
    "Pre_A": "xkcd:light red",
    "Post_HumCan": "xkcd:green",
    "Post_MED": "xkcd:blue",
    "Post_A": "xkcd:red",
    "TD_HumCan": "xkcd:dark green",
    "TD_MED": "xkcd:dark blue",
    "TD_A": "xkcd:dark red",
}
linestyles = {
    "Mean": "--",
    "Median": "-.",
}

fig, axes = plt.subplots(
    3, 1, figsize=(4, 10), sharex=True, gridspec_kw=dict(hspace=0.05)
)
axes = axes.flatten()
sns.despine(fig)
for pcmodel_sample in list(solutions):
    op, group = pcmodel_sample.replace(f"{pcmodel.id}_", "").split("_", maxsplit=1)
    color = colors.get(group, "xkcd:black")
    linestyle = linestyles.get(op, ":")
    s_values = solutions[str(pcmodel_sample)]["slack"].values
    r_values = solutions[str(pcmodel_sample)]["relaxation"].values
    o_values = solutions[str(pcmodel_sample)]["objective"].values
    rxn_values = solutions[str(pcmodel_sample)]["_".join(objective_reactions)].values

    zorder = 1
    lw = 2
    axes[0].plot(
        s_values,
        r_values,
        label=str(pcmodel_sample),
        color=color,
        linestyle=linestyle,
        linewidth=lw,
        zorder=zorder,
    )
    axes[1].plot(
        s_values,
        o_values,
        label=str(pcmodel_sample),
        color=color,
        linestyle=linestyle,
        linewidth=lw,
        zorder=zorder,
    )
    axes[2].plot(
        s_values,
        rxn_values,
        label=str(pcmodel_sample),
        color=color,
        linestyle=linestyle,
        linewidth=lw,
        zorder=zorder,
    )

fontdict = {"size": "x-large"}
axes[-1].set_xlabel(r"Slack variable $s$", fontdict=fontdict)

zorder = 2
alpha = 0.7
limit_pad_sclar = 1.2
smin = s_values[0]

for i, ax in enumerate(axes):
    if i == 0:
        ymin, ymax = (
            -0.001,
            max(r_values) * limit_pad_sclar,
        )
    elif i == 1:
        ymin, ymax = (
            min(0, min(o_values) * 2) * limit_pad_sclar,
            max(o_values) * limit_pad_sclar,
        )
    elif i == 2:
        ymin, ymax = (0, max(rxn_values) * limit_pad_sclar)
    else:
        pass
    ax.vlines(chosen_slack_var, ymin=ymin, ymax=ymax, color="black", linestyle=":")
    ax.vlines(
        smin,
        ymin=ymin,
        ymax=ymax,
        color="black",
        linestyle="-",
        zorder=zorder,
        alpha=alpha,
    )
    ax.vlines(
        1,
        ymin=ymin,
        ymax=ymax,
        color="black",
        linestyle="-",
        zorder=zorder,
        alpha=alpha,
    )
    ax.set_xlim(smin / 2, slack_max)
    ax.set_ylim(ymin, ymax)
    ax.set_xscale("log")
    if i == 0:
        ax.annotate(
            rf"$s > 0$",
            xy=(smin, ymax),
            xycoords="data",
            xytext=(10, 5),
            textcoords="offset points",
            ha="center",
            fontsize=fontdict["size"],
        )
        ax.annotate(
            rf"$s \leq 1$",
            xy=(1, ymax),
            xycoords="data",
            xytext=(10, 5),
            textcoords="offset points",
            ha="center",
            fontsize=fontdict["size"],
        )
        ax.annotate(
            rf"$s = {chosen_slack_var}$",
            xy=(chosen_slack_var, 0),
            xycoords="data",
            xytext=(5, 0),
            textcoords="offset points",
            ha="left",
            fontsize=fontdict["size"],
        )
    ax.fill_between((smin / 2, smin), ymin, ymax, color="xkcd:light grey")
    ax.annotate(
        "Infeasible",
        xy=(smin, (ymax + ymin) / 2),
        xycoords="data",
        rotation=90,
        xytext=(-2, 0),
        textcoords="offset points",
        va="center",
        ha="right",
        fontsize=fontdict["size"],
    )


handles, labels = axes[2].get_legend_handles_labels()
labels = [label.replace(f"{pcmodel.id}_", "").replace("_", " ") for label in labels]
axes[2].legend(
    handles=handles,
    labels=labels,
    ncols=2,
    frameon=False,
    loc="upper center",
    fontsize="large",
    bbox_to_anchor=(0.5, -0.2),
)


axes[0].set_ylabel("Relaxation budget used (%)", fontdict=fontdict)
axes[1].set_ylabel("Objective value", fontdict=fontdict)
axes[2].set_ylabel(f"{'+'.join(objective_reactions)} (mmol/gDW/hr)", fontdict=fontdict)

fig.align_labels()
if save_figures:
    fig.savefig(
        fitting_dirpath / f"SlackPercentDetermination_{model_id}.{imagetype}",
        transparent=transparent,
        format=imagetype,
    )

### Formulate models from QP solutions

In [ ]:
# SBML/XML loads faster, but will take up to 4x more space uncompressed as compared to JSON
ftypes = {
    "xml"
    # "json",
}

slack_value = chosen_slack_var  # Slack %
ftypes = set([ftypes]) if isinstance(ftypes, str) else set(ftypes)
for sample_id, (df_qp_sol, objective_value, r2_values) in qp_solutions_dict.items():
    # Create a copy of the model
    sample_id = f"{pcmodel.id}_{sample_id}"
    filenames = [sample_pcmodels_dirpath / f"{sample_id}.{ftype}" for ftype in ftypes]
    if not overwrite and all([filename.exists() for filename in filenames]):
        print(f"Model already created for {sample_id}")
        continue
    pcmodel_sample = pcmodel.copy()
    pcmodel_sample.id = sample_id
    for protdl in pcmodel_sample.reactions.query(
        lambda x: isinstance(x, ProteinDilution)
    ):
        if protdl.id in df_qp_sol.index:
            prot_bound = df_qp_sol.loc[protdl.id]["Best-Fitted Proteome"]
        else:
            prot_bound = 0
        protdl.bounds = (float(prot_bound), float(prot_bound))
    # Add the relaxation budget
    add_relaxation_budget(pcmodel_sample, slack_value, verbose)
    budget_rxn_relaxation = pcmodel_sample.reactions.get_by_id(
        f"{budget_rxn_prefix}{budget_met_prefix}relaxation"
    )
    with pcmodel_sample:
        pcmodel_sample.objective = budget_rxn_relaxation.flux_expression
        pcmodel_sample.objective_direction = "min"
        budget_min = pcmodel_sample.slim_optimize()
        pcmodel_sample.objective_direction = "max"
        budget_max = pcmodel_sample.slim_optimize()
    budget_rxn_relaxation.bounds = (budget_min, budget_max)
    # # Store model for later use
    # list_of_relaxed_models += [pcmodel_sample]
    for filename in filenames:
        # Might as well overwrite all files, especially if model needed to be regenerated anyways
        write_cobra_model(
            pcmodel_sample,
            filename=filename,
        )